In [28]:
#importing libraries

import re
import random
import math
import collections
import itertools

errors=0

# data initialization

def read_files(sentiment_dictionary, sentences_train, sentences_test):
  pos_sent=open('rt-polarity.pos', 'r', encoding='ISO-8859-1')
  pos_sent=re.split(r'\n', pos_sent.read())

  neg_sent=open('rt-polarity.neg', 'r', encoding='ISO-8859-1')
  neg_sent=re.split(r'\n', neg_sent.read())

  pos_dictionary = open('positive-words.txt', 'r', encoding="ISO-8859-1")
  pos_wordlist = pos_dictionary.readlines()
  pos_wordlist = [line.strip() for line in pos_wordlist if not line.startswith(";") and not line == '\n']
 
  neg_dictionary = open('negative-words.txt', 'r', encoding="ISO-8859-1")
  neg_wordlist = neg_dictionary.readlines()
  neg_wordlist = [line.strip() for line in neg_wordlist if not line.startswith(";") and not line == '\n']

  for i in pos_wordlist:
        sentiment_dictionary[i] = 1
  for i in neg_wordlist:
        sentiment_dictionary[i] = -1

  for i in pos_sent:
        if random.randint(1,10)<2:
            sentences_test[i]="positive"
        else:
            sentences_train[i]="positive"

  for i in neg_sent:
        if random.randint(1,10)<2:
            sentences_test[i]="negative"
        else:
            sentences_train[i]="negative"



In [29]:

def testBayes(sentences_test, dataName, pWordPos, pWordNeg, pWord,pPos):
    pNeg=1-pPos

    total=0
    correct=0
    totalpos=0
    totalpospred=0
    totalneg=0
    totalnegpred=0
    correctpos=0
    correctneg=0

    for sentence, sentiment in sentences_test.items():
        wordList = re.findall(r"[\w']+", sentence)

        bigramList=wordList.copy() #initialise bigramList
        for x in range(len(wordList)-1):
           bigramList.append(wordList[x]+"_" + wordList[x+1])

        trigramList=bigramList     #initialise trigtamList
        for x in range(len(wordList)-2):
            trigramList.append(wordList[x]+"_"+wordList[x+1]+"_"+wordList[x+2])
        
        
        pPosW=pPos
        pNegW=pNeg


        for word in bigramList: #calculate over bigrams
            if word in pWord:
                if pWord[word]>0.00000001:
                    pPosW *=pWordPos[word]
                    pNegW *=pWordNeg[word]

        prob=0;            
        if pPosW+pNegW >0:
            prob=pPosW/float(pPosW+pNegW)

        total+=1
        if sentiment=="positive":
            totalpos+=1
            if prob>0.5:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
                if errors:
                    print ("ERROR (pos classed as neg %0.2f):" %prob + sentence)
        else:
            totalneg+=1
            if prob<=0.5:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
                if errors:
                    print ("ERROR (neg classed as pos %0.2f):" %prob + sentence)
    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")")

    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred)
    recall_neg=correctneg/float(totalneg)
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg);

    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg)
 

In [30]:
def test_dictionary(sentences_test, dataName, sentiment_dictionary, threshold):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentences_test.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        for word in Words:
            if word in sentiment_dictionary:
               score+=sentiment_dictionary[word]
 
        total+=1
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
 
    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + "(%d" % correct + "/%d" % total +")")
    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred)
    recall_neg=correctneg/float(totalneg)
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg);


    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg )

def mostUseful(pWordPos, pWordNeg, pWord, n):
    predictPower={}
    for word in pWord:
        if pWordNeg[word]<0.0000001:
            predictPower[word]=1000000000
        else:
            predictPower[word]=pWordPos[word] / (pWordPos[word] + pWordNeg[word])

    sortedPower = sorted(predictPower, key=predictPower.get)
    head, tail = sortedPower[:n], sortedPower[len(predictPower)-n:]
    print ("\nNEGATIVE:")
    print (head)
    print ("\nPOSITIVE:")
    print (tail)

In [31]:
sentiment_dictionary={} # {} initialises a dictionary [hash function]
sentences_train={}
sentences_test={}

#initialise datasets and dictionaries
read_files(sentiment_dictionary,sentences_train,sentences_test) 

pWordPos={} # p(W|Positive)
pWordNeg={} # p(W|Negative)
pWord={}    # p(W) 

#build conditional probabilities using training data
trainBayes(sentences_train, pWordPos, pWordNeg, pWord)

#run naive bayes classifier on datasets
print ("\nNaive Bayes")
testBayes(sentences_train,  "\nFilms (Train Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentences_test,  "\nFilms  (Test Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)

# print most useful words
mostUseful(pWordPos, pWordNeg, pWord, 50)




['enigma', 'is', 'well', 'made', 'but', "it's", 'just', 'too', 'dry', 'and', 'too', 'placid', 'enigma_is', 'is_well', 'well_made', 'made_but', "but_it's", "it's_just", 'just_too', 'too_dry', 'dry_and', 'and_too', 'too_placid', 'enigma_is_well', 'is_well_made', 'well_made_but', "made_but_it's", "but_it's_just", "it's_just_too", 'just_too_dry', 'too_dry_and', 'dry_and_too', 'and_too_placid']

Naive Bayes

Films (Train Data, Naive Bayes)	 Accuracy (All)=0.82 (7844/9595)

Films (Train Data, Naive Bayes)	 Precision (Pos)=0.93 (3300/3564)

Films (Train Data, Naive Bayes)	 Recall (Pos)=0.69 (3300/4787)

Films (Train Data, Naive Bayes)	 F-measure (Pos)=0.79

Films (Train Data, Naive Bayes)	 Precision (Neg)=0.75 (4544/6031)

Films (Train Data, Naive Bayes)	 Recall (Neg)=0.95 (4544/4808)

Films (Train Data, Naive Bayes)	 F-measure (Neg)=0.84

Films  (Test Data, Naive Bayes)	 Accuracy (All)=0.80 (851/1068)

Films  (Test Data, Naive Bayes)	 Precision (Pos)=0.80 (436/545)

Films  (Test Data, Naiv